# Tensorboard Embedding Visualization

In [5]:
import gensim
import tensorflow as tf
import numpy as np
from tensorflow.contrib.tensorboard.plugins import projector
import os
from health.codes import *
import sys
import pickle

reload(sys)
sys.setdefaultencoding('utf-8')

In [6]:
from health.codes import *

In [22]:
code2vecDict = {}
model = gensim.models.Word2Vec.load('code2vec.p')
good_keys = []
for key, val in model.wv.vocab.iteritems():
    code2vecDict[key] = model[key]

In [23]:
pickle.dump(code2vecDict, open('code2vecDict.p','wb'))

In [52]:
embeddings = np.zeros((8418, 100))

words = []
for i, key in enumerate(good_keys):
    embeddings[i,:] = model[key]
    words.append(key)

In [61]:
LOG_DIR = './tf-logs'
session = tf.Session()
embedding_var = tf.Variable(embeddings)
sess.run(embedding_var.initializer)
summary_writer = tf.summary.FileWriter(LOG_DIR)
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
embedding.tensor_name = embedding_var.name

# Comment out if you don't have metadata
embedding.metadata_path = os.path.join(LOG_DIR, 'metadata.tsv')

# Comment out if you don't want sprites
#embedding.sprite.image_path = os.path.join(LOG_DIR, 'sprite.png')
#embedding.sprite.single_image_dim.extend([imgs.shape[1], imgs.shape[1]])

projector.visualize_embeddings(summary_writer, config)
saver = tf.train.Saver([embedding_var])
saver.save(sess, os.path.join(LOG_DIR, 'model2.ckpt'), 1)

'./tf-logs/model2.ckpt-1'

In [62]:
metadata_file = open(os.path.join(LOG_DIR, 'metadata.tsv'), 'w')
metadata_file.write('Description\tCCS Group\tICD9\tCCS\n')
for i,code in enumerate(good_keys):
    metadata_file.write('%s\t%s\t%s\t%s\n' % (icd2desc[code], ccs2desc[icd2ccs[code]], code, icd2ccs[code]))
metadata_file.close()

In [9]:
saver = tf.train.Saver()
saver.save(session, os.path.join(LOG_DIR, "model.ckpt"), step)

# Use the same LOG_DIR where you stored your checkpoint.
summary_writer = tf.train.SummaryWriter(LOG_DIR)

# Format: tensorflow/contrib/tensorboard/plugins/projector/projector_config.proto
config = projector.ProjectorConfig()

# You can add multiple embeddings. Here we add only one.
embedding = config.embeddings.add()
embedding.tensor_name = embedding_var.name
# Link this tensor to its metadata file (e.g. labels).
embedding.metadata_path = os.path.join(LOG_DIR, 'metadata.tsv')

# Saves a configuration file that TensorBoard will read during startup.
projector.visualize_embeddings(summary_writer, config)

NameError: name 'step' is not defined